In [17]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv('../data/real_0_0.csv').sort_values('logB', ascending=False)

In [24]:
def _twin(df, df_query):
    
    def similarity(row):
        # props = ['T', 'data_points']
        # x = np.array(row[props])
        # y = np.array(df_query[props])
        # return np.sqrt(np.sum(np.square(x - y) / np.square(y)))
        d1 = np.abs(row['T'] - df_query['T']) / df_query['T']
        d2 = np.abs(row['data_points'] - df_query['data_points']) / df_query['data_points']
        return np.max([d1, d2])
        
    sim = np.array(df.apply(similarity, axis= 1))
    best_match = np.argmin(sim)
    return {'best_match': df.iloc[best_match]['id']}
    
    
def twin(df, num):
    """Remove the top 'num' candidates from 'df'. For each removed candidate the most similar quasar among the remaining quasars is found. 
       Returns: an array of removed candidates with attached id of their twin. 
    """
    removed = df.iloc[:num]
    remaining = df.iloc[num:]
    
    best_match = removed.apply(lambda df_query: _twin(remaining, df_query), axis= 1, result_type= 'expand')
    removed = removed.join(best_match)
    return removed
    

removed= twin(data, 30)

removed.to_csv('../data/twins.csv')